In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize,differential_evolution
from BatteryV3 import *
import matplotlib.pyplot as plt

# ==========================================
# 2. 数据加载与预处理
# ==========================================
try:
    df = pd.read_csv(r'C:\Users\lenovo\Desktop\archive\cleaned_dataset\discharge\1.csv')
    print("已加载数据。")
except:
    print("未找到文件，生成测试数据...")
    df = pd.DataFrame({'Time': np.linspace(0,3600,100), 'Voltage_measured': np.linspace(4.2,3.2,100), 'Current_measured': [-1.0]*100, 'Temperature_measured': [24.0]*100})

time_data = df['Time'].values
current_data = -df['Current_measured'].values # 修正符号
voltage_data = df['Voltage_measured'].values
temp_data = df['Temperature_measured'].values
dt_data = np.zeros_like(time_data)
dt_data[1:] = np.diff(time_data)
dt_data[0] = dt_data[1] if len(dt_data) > 1 else 1.0

# ==========================================
# 3. 优化核心逻辑
# ==========================================
def run_sim(params):
    # Unpack params
    p_E0, p_R, p_K, p_A, p_B, p_Cap, p_InitSOC = params
    
    batt = PhysicsBattery(design_capacity_ah=p_Cap, initial_soc=p_InitSOC, initial_temp_c=temp_data[0])
    batt.E0 = p_E0
    batt.R_base = p_R
    batt.K = p_K
    batt.A_vol = p_A
    batt.B_vol = p_B

    
    v_list = []
    t_env = temp_data[0]
    
    # Fast loop
    for i in range(len(time_data)):
        v = batt.step(current_data[i], dt_data[i], temp_env_c=t_env)
        v_list.append(v)
    return np.array(v_list)

def loss_func(params):
    # 物理约束惩罚
    if params[5] <= 0.1: return 1e6 # Capacity
    if params[0] < 2.0: return 1e6 # E0
    
    try:
        sim_res = run_sim(params)
        sim_v = sim_res[:, 0]
        
        mask = ~np.isnan(sim_v)
        if np.sum(mask) < len(sim_v) * 0.9: return 1e6
        
        diff = sim_v[mask] - voltage_data[mask]
        
        # === 新增：加权 MSE ===
        # 给电压低于 3.5V 的区域更高的权重（例如 5倍权重）
        # 这样迫使模型必须拟合好最后的下降曲线
        weights = np.ones_like(diff)
        weights[voltage_data[mask] < 3.5] = 5.0 
        
        weighted_mse = np.mean(weights * (diff**2))
        return np.sqrt(weighted_mse)
        # ====================
        
    except:
        return 1e6

# ==========================================
# 4. 两阶段优化策略
# ==========================================

# --- 阶段 1: 扩大范围的全局搜索 ---
# 我们大幅放宽了边界，尤其是 R 和 E0
bounds_wide = [
    (3.2, 4.5),    # E0: 允许更低的 OCV (有些老化电池 OCV 很低)
    (0.001, 0.5),  # R_base: 允许非常大的内阻 (1欧姆对于老化电池是可能的)
    (0.0, 0.1),    # K
    (0.0, 2.0),    # A_vol: 允许更大的初始电压降
    (0.1, 20.0),   # B_vol: 允许更陡峭的指数曲线
    (1.0, 3.0),    # Capacity
    (0.99, 1.0)    # Init SOC: 允许电池从半电开始

]


print(">>> 阶段 1: 全局搜索 (Differential Evolution)...")
result_stage1 = differential_evolution(
    loss_func, 
    bounds_wide, 
    strategy='best1bin', 
    maxiter=30, 
    popsize=15, 
    tol=0.05, 
    disp=True
)
print(f"阶段 1 完成. RMSE: {result_stage1.fun:.5f} V")

# --- 阶段 2: 局部精细微调 ---
print("\n>>> 阶段 2: 局部微调 (Nelder-Mead)...")
# 使用阶段 1 的结果作为起点
result_stage2 = minimize(
    loss_func, 
    result_stage1.x, 
    method='Nelder-Mead', 
    tol=1e-5,
    options={'maxiter': 500, 'disp': True}
)

final_params = result_stage2.x
print(f"\n最终优化完成! RMSE: {result_stage2.fun:.5f} V")
print("最佳参数:", final_params)

# ==========================================
# 5. 绘图与误差分析 (修正版：包含自动计算的参考 SOC)
# ==========================================
# 1. 运行最终模拟并拆分结果
sim_res_final = run_sim(final_params)
sim_final_v = sim_res_final[:, 0]    # 提取模型预测电压
sim_final_soc = sim_res_final[:, 1]  # 提取模型预测 SOC

# --- 新增内容：基于库仑计数法计算“参考 SOC” (Ground Truth) ---
# 使用优化出的容量 (final_params[5]) 和初始 SOC (final_params[6]) 作为计算基准
opt_cap_ah = final_params[5]
opt_init_soc = final_params[6]

# 计算每一时刻消耗的库仑量 (安培·秒)
# current_data 已经是正值（放电），dt_data 是步长
discharged_coulombs = np.cumsum(current_data * dt_data) 
# 计算参考 SOC 轨迹
ref_soc_coulomb = opt_init_soc - (discharged_coulombs / (opt_cap_ah * 3600))
# -------------------------------------------------------

plt.figure(figsize=(10, 6))

# 2. 画出真实电压数据
plt.plot(time_data, voltage_data, 'k-', linewidth=1.5, alpha=0.7, label='Measured Voltage')

# 3. 画出模拟的电压数据 (使用修正后的 sim_final_v)
plt.plot(time_data, sim_final_v, 'r--', linewidth=2, label='Fitted Model (Voltage)')

# 4. 标注误差区域
plt.fill_between(time_data, voltage_data, sim_final_v, color='yellow', alpha=0.3, label='Error Area')

# 5. 设置双 Y 轴来显示 SOC 变化
ax2 = plt.gca().twinx()

# 画出模型内部 KiBaM 动力学计算的 SOC (蓝色点线)
ax2.plot(time_data, sim_final_soc * 100, 'b:', alpha=0.8, label='Predicted SOC (%)')

# 画出通过库仑计数法计算出的“参考 SOC” (绿色虚线)
# 这代表了基于电流积分的电荷平衡，是模型应当对齐的目标
#ax2.plot(time_data, ref_soc_coulomb * 100, 'g--', linewidth=1.5, alpha=0.8, label='Reference SOC (%)')

ax2.set_ylabel('State of Charge (%)', color='blue')
ax2.tick_params(axis='y', labelcolor='blue')
ax2.set_ylim(0, 105)

plt.title(f'High-Precision Fit & SOC Comparison (RMSE: {result_stage2.fun:.4f} V)')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')

# 合并两个轴的图例
lines, labels = plt.gca().get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
plt.legend(lines + lines2, labels + labels2, loc='upper right')

plt.grid(True, linestyle=':', alpha=0.6)

# 打印最终优化参数
param_str = (
    f"E0={final_params[0]:.3f}\n"
    f"R ={final_params[1]:.3f}\n"
    f"Cap={final_params[5]:.3f}\n"
    f"SOC={final_params[6]:.2f}"
)
plt.text(0.02, 0.05, param_str, transform=plt.gca().transAxes, 
          bbox=dict(facecolor='white', alpha=0.9))

plt.savefig('final_best_fit.png')
print("图像已保存至 final_best_fit.png")

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from BatteryV3 import *
import os

# 初始化 (建议使用拟合出的参数)
battery = PhysicsBattery() 

def run_adaptive_simulation(model, file_path, learning_rate_R=1e-5, feedback_gain_soc=0.05):
    """
    自适应仿真：
    1. 使用电压误差实时修正 SOC (Observer)
    2. 使用平均误差动态更新内阻 R (Parameter Estimation)
    """
    df = pd.read_csv(file_path)
    df['dt'] = df['Time'].diff().fillna(0)
    
    v_preds = []
    soc_preds = []
    r_history = []  # 记录内阻变化
    
    # 周期内的误差累积，用于更新内阻
    cycle_voltage_error_sum = 0
    valid_points_count = 0
    
    for i, row in df.iterrows():
        # 1. 获取输入
        current = -row['Current_measured'] # 你的数据: 负数为放电, 模型: 正数为放电
        dt = row['dt']
        temp = row['Temperature_measured']
        v_meas = row['Voltage_measured']
        
        # 2. 先进行开环预测 (Open Loop Prediction)
        #    注意：这里我们先不让 step 更新状态，只是为了算个预测电压
        #    (由于 BatteryV3 耦合较紧，我们这里先正常 step，然后用反馈修正回去)
        v_est, soc_est = model.step(current, dt, temp_env_c=temp)
        
        # 3. 计算新息 (Innovation/Error)
        #    模型算出来的电压 vs 真实电压
        voltage_error = v_meas - v_est
        
        # ---------------------------------------------------------
        # 闭环策略 A: 软性 SOC 修正 (Soft SOC Nudging)
        # ---------------------------------------------------------
        # 如果模型电压 < 真实电压，说明模型 SOC 估低了，需要加一点
        # 如果模型电压 > 真实电压，说明模型 SOC 估高了，需要减一点
        # 只有在电流较大时(电压包含SOC信息)且非剧烈变化时才修正
        if abs(current) > 0.1 and i > 10: 
            # 增益 K 需要非常小，否则会震荡
            # 反馈量 = 增益 * 电压误差 * dt
            # 注意符号：SOC高 -> OCV高 -> V_est高 -> Error负 -> SOC应减小。方向正确。
            
            soc_correction = feedback_gain_soc * voltage_error * dt
            
            # 将修正量应用回模型的内部状态 (y1, y2)
            # 按比例分配给 bound/available charge
            total_charge_old = model.y1 + model.y2
            total_charge_new = total_charge_old + (soc_correction * model.capacity_design_c)
            
            # 防止修正出界
            total_charge_new = max(0, min(total_charge_new, model.capacity_design_c))
            
            if total_charge_old > 0:
                ratio = total_charge_new / total_charge_old
                model.y1 *= ratio
                model.y2 *= ratio
        
        # ---------------------------------------------------------
        # 闭环策略 B: 在线内阻学习 (Online R Learning)
        # ---------------------------------------------------------
        # 简单的 LMS (Least Mean Squares) 梯度下降
        # 如果 V_est 总是比 V_meas 大 (预测偏高)，说明还需要更大的压降 -> R 需要变大
        # 只有在放电时更新 R
        if current > 0.5: 
            # 梯度方向：Error = V_meas - (OCV - I*R)
            # d(Error)/dR = I
            # R_new = R_old + alpha * Error * I
            # 这里的符号：如果 V_meas < V_est (Error < 0)，说明真实掉电快，内阻其实更大
            # 所以我们要 增加 R。
            # update = - learning_rate * error * current
            
            r_update = - learning_rate_R * voltage_error * abs(current)
            model.R_base += r_update
            
            # 物理约束：内阻不能是负的，也不能无限大
            model.R_base = max(0.01, min(model.R_base, 2.0))

        
        # 记录数据
        v_preds.append(v_est)
        soc_preds.append(model.y1 / (model.c * model.capacity_design_c)) # 记录 y1 对应的瞬时 SOC
        r_history.append(model.R_base)

    df['V_pred'] = v_preds
    df['SOC_pred'] = soc_preds
    df['R_adaptive'] = r_history
    
    return df

# ==========================================
# 主运行逻辑
# ==========================================
save_path = r"C:\Users\lenovo\Desktop\archive\old\test_now\pictures"
dis_base_path = r"C:\Users\lenovo\Desktop\archive\cleaned_dataset\discharge"
in_base_path = r"C:\Users\lenovo\Desktop\archive\cleaned_dataset\charge"

# 参数调优建议：
# learning_rate_R: 内阻更新速率。太大会震荡，太小跟不上老化。建议 1e-4 到 1e-6
# feedback_gain_soc: SOC 修正增益。建议 0.01 到 0.1
LR_R = 5e-5 
GAIN_SOC = 0.005 

for i in range(1, 73):
    print(f"--- Cycle {i} ---")
    
    # 1. 跑充电 (允许模型在充电时也进行一定程度的自适应，或者只跑不修)
    #    充电时通常电流较小，内阻估算不准，建议在充电时关闭内阻更新 (learning_rate_R=0)
    charge_path = os.path.join(in_base_path, f"{i-1}.csv")
    if os.path.exists(charge_path):
        run_adaptive_simulation(battery, charge_path, learning_rate_R=0, feedback_gain_soc=GAIN_SOC)
        # 充电结束，仍然建议做一个软校准，但不要强制设为1.0
        # 如果电压 > 4.15，说明确实满了，可以让 SOC 缓慢逼近 1.0
        if battery.history['voltage'][-1] > 4.15:
            # 软复位：只消除 50% 的误差，保留一部分内部状态
            current_q = battery.y1 + battery.y2
            target_q = battery.capacity_design_c
            new_q = current_q * 0.5 + target_q * 0.5 
            ratio = new_q / current_q
            battery.y1 *= ratio
            battery.y2 *= ratio
            print("  [Soft Reset] Charge complete. Partially aligned SOC to 100%.")

    # 2. 跑放电 (开启全功能自适应)
    dis_path = os.path.join(dis_base_path, f"{i}.csv")
    if os.path.exists(dis_path):
        df_res = run_adaptive_simulation(battery, dis_path, learning_rate_R=LR_R, feedback_gain_soc=GAIN_SOC)
        
        final_R = df_res['R_adaptive'].iloc[-1]
        print(f"  Discharge {i} Complete. Adaptive R_base: {final_R:.4f} Ohm")
        
        # 绘图检查
        plt.figure(figsize=(10, 6))
        
        plt.subplot(2, 1, 1)
        plt.plot(df_res['Time'], df_res['Voltage_measured'], 'k', label='Actual', alpha=0.6)
        plt.plot(df_res['Time'], df_res['V_pred'], 'r--', label='Adaptive Model')
        plt.title(f'Cycle {i}: Voltage Fit (R={final_R:.3f})')
        plt.legend()
        
        plt.subplot(2, 1, 2)
        plt.plot(df_res['Time'], df_res['R_adaptive'], 'g')
        plt.title('Internal Resistance Adaptation')
        plt.ylabel('R_base (Ohms)')
        plt.xlabel('Time (s)')
        
        plt.tight_layout()
        plt.savefig(os.path.join(save_path, f"{i}.png"))
        plt.close()

--- Cycle 1 ---
  Discharge 1 Complete. Adaptive R_base: 0.2023 Ohm
--- Cycle 2 ---
  [Soft Reset] Charge complete. Partially aligned SOC to 100%.
  Discharge 2 Complete. Adaptive R_base: 0.2024 Ohm
--- Cycle 3 ---
  [Soft Reset] Charge complete. Partially aligned SOC to 100%.
  Discharge 3 Complete. Adaptive R_base: 0.2025 Ohm
--- Cycle 4 ---
  [Soft Reset] Charge complete. Partially aligned SOC to 100%.
  Discharge 4 Complete. Adaptive R_base: 0.2027 Ohm
--- Cycle 5 ---
  [Soft Reset] Charge complete. Partially aligned SOC to 100%.
  Discharge 5 Complete. Adaptive R_base: 0.2028 Ohm
--- Cycle 6 ---
  [Soft Reset] Charge complete. Partially aligned SOC to 100%.
  Discharge 6 Complete. Adaptive R_base: 0.2030 Ohm
--- Cycle 7 ---
  [Soft Reset] Charge complete. Partially aligned SOC to 100%.
  Discharge 7 Complete. Adaptive R_base: 0.2031 Ohm
--- Cycle 8 ---
  [Soft Reset] Charge complete. Partially aligned SOC to 100%.
  Discharge 8 Complete. Adaptive R_base: 0.2033 Ohm
--- Cycle 9 ---